# Incontext Approach Description


- We have a folder named PUBLIC SAMPLES with two sub-folders Cases and Controls
- The two subfolders only contain audio .mp3 files and their file names.
- So we get the data from the subfolders with columns
  - file Name
  - Sampling Rate (Hz)
  - Channels
  - Duration (seconds)
- Then we label the folders by creating 2 columns
  - folder: Original folder location (Cases or Controls)
  - hypernasality: 0 for no hypernasality and 1 for yes hypernasality
- Then we parse the file names to extract the original text or phrase that the individuals have to talk and this would be the original_text column
- Then we process the audio files through Openai's Whisper model as GPT4 turbo's api can't preprocess the audio files.
- This creates a text representation of the audio files in the OPENAI_Whisper_text column of how the whisper model interprets the text.
- Then we concat the cases and control dataframes into 1 dataframe and make it into a excel file for future access:  output_filename.xlsx
- We then set the Gpt4 turbo model up
- Then we split the data into test and train by 8:2.
- Then we use incontext learning by using examples from the training set and use it to identify the test set
- This is more clear if you see the train_prompt and test_prompt in the bottom.
- Based on the test results, we got a 50% accuracy (5 correct out of 10)
- This is just a initial set up and we could probably increase the accuracy with different prompting.

# packages

In [ ]:
from google.colab import drive
import pandas as pd

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install librosa soundfile
!pip install mutagen
!pip install librosa pydub
!pip install openai

# Database


In [ ]:
# Step 2: Import necessary libraries
import os
from pydub import AudioSegment
import librosa
import pandas as pd

# Define the folder path
folder_path = '/content/drive/MyDrive/VUMC_postcleft/PUBLIC SAMPLES/CASES'

# List all files in the directory
files = os.listdir(folder_path)

# Filter out MP3 files
mp3_files = [file for file in files if file.endswith('.mp3')]

# Modify the function to return audio file properties instead of printing
def analyze_audio_file(file_name):
    file_path = os.path.join(folder_path, file_name)

    # Using pydub to check the number of channels
    audio_segment = AudioSegment.from_file(file_path)
    channels = audio_segment.channels

    # Using librosa to load audio and check the sampling rate
    audio, sample_rate = librosa.load(file_path, sr=None)  # Load with original sample rate
    duration = librosa.get_duration(y=audio, sr=sample_rate)

    return {
        "File Name": file_name,
        "Sampling Rate (Hz)": sample_rate,
        "Channels": channels,
        "Duration (seconds)": round(duration, 2)
    }

# Analyze all MP3 files and collect the data
audio_files_data = [analyze_audio_file(file) for file in mp3_files]
cases_df = pd.DataFrame(audio_files_data)
cases_df['folder'] = 'CASES'
cases_df['hypernasality'] = 1

cases_df

,File Name,Sampling Rate (Hz),Channels,Duration (seconds),folder,hypernasality
0,"ACPA 2 more in grammar school, and out little ...",44100,1,3.27,CASES,1
1,ACPA and I started out on the switch board and...,44100,1,3.89,CASES,1
2,ACPA playing in the snow is fun-3.mp3,48000,1,2.18,CASES,1
3,Facebook (pick up the baby).mp3,44100,1,2.51,CASES,1
4,"ACPA oh let me see, what can I tell you about ...",44100,1,4.13,CASES,1
...,...,...,...,...,...,...
91,Video 3_5 (muddy).mp3,44100,2,1.20,CASES,1
92,Video 6_1 (mom).mp3,44100,2,0.99,CASES,1
93,Video 1_4 (seesaw).mp3,44100,2,1.15,CASES,1
94,Video 6_5 (bobby).mp3,44100,2,0.84,CASES,1


In [ ]:
# Step 2: Import necessary libraries
import os
import pandas as pd
from mutagen.mp3 import MP3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 3: Define the folder path
folder_path = '/content/drive/MyDrive/VUMC_postcleft/PUBLIC SAMPLES/CONTROLS'

# Step 4: List all files in the directory
files = os.listdir(folder_path)

# Step 4: List all files in the directory
files = os.listdir(folder_path)


# Step 5: Filter out MP3 files
mp3_files = [file for file in files if file.endswith('.mp3')]

# Step 6: Get file size and duration for each file
file_info = []
for file in mp3_files:
    path = os.path.join(folder_path, file)
    file_size = os.path.getsize(path)  # File size in bytes
    audio = MP3(path)
    duration = round(audio.info.length, 1)  # Duration in seconds, rounded to one decimal place

    file_info.append({'File Name': file, 'File Size (bytes)': file_size, 'Duration (seconds)': duration})

# Step 7: Create a DataFrame
controls_df = pd.DataFrame(file_info)
controls_df['File Size (MB)'] = controls_df['File Size (bytes)'] / (1024 * 1024)

controls_df
control_df = pd.DataFrame(audio_files_data)
control_df['folder'] = 'CONTROLS'
control_df['hypernasality'] = 0

control_df


,File Name,Sampling Rate (Hz),Channels,Duration (seconds),folder,hypernasality
0,ACPA I like school.mp3,44100,1,2.35,CONTROLS,0
1,ACPA she wore blue shoes-2.mp3,44100,1,2.14,CONTROLS,0
2,ACPA they feed seeds to birds.mp3,44100,1,2.98,CONTROLS,0
3,ACPA Most Boys Like to Play Football.mp3,44100,1,3.06,CONTROLS,0
4,ACPA Ted had a dog with white feet.mp3,44100,1,4.18,CONTROLS,0
...,...,...,...,...,...,...
83,Video 1_6 (good).mp3,44100,2,0.76,CONTROLS,0
84,Video 1_4 (and can I have some more material).mp3,44100,2,3.60,CONTROLS,0
85,Video 1_5 (do you have a knife).mp3,44100,2,1.70,CONTROLS,0
86,Video 1_7 (here_s some pizza).mp3,44100,2,2.12,CONTROLS,0


In [ ]:
############## Adding two dfs
result_df = pd.concat([cases_df, control_df], ignore_index=True)

###########Chaning the original_text
result_df['original_text'] = result_df['File Name']
result_df['original_text'] = result_df['original_text'].str.replace('.mp3', '', regex=False)
# Remove 'APCA' from each value in 'original_text'
result_df['original_text'] = result_df['original_text'].str.replace('ACPA', '', regex=False)
# Trim leading and trailing spaces
result_df['original_text'] = result_df['original_text'].str.strip()
result_df['original_text'] = result_df['original_text'].str.extract(r'\((.*?)\)', expand=False).fillna(result_df['original_text'])
#
result_df['original_text'] = result_df['original_text'].str.replace(r'-\d+', '', regex=True)
result_df['original_text'] = result_df['original_text'].str.replace('_', "'", regex=False)

########## Whisper text
result_df['OPENAI_Whisper_text'] = ''

########## Renaming
result_df.columns = result_df.columns.str.replace(' ', '_')

# Final df
result_df

,File_Name,Sampling_Rate_(Hz),Channels,Duration_(seconds),folder,hypernasality,original_text,OPENAI_Whisper_text
0,"ACPA 2 more in grammar school, and out little ...",44100,1,3.27,CASES,1,"2 more in grammar school, and out little 3 yea...",
1,ACPA and I started out on the switch board and...,44100,1,3.89,CASES,1,and I started out on the switch board and two ...,
2,ACPA playing in the snow is fun-3.mp3,48000,1,2.18,CASES,1,playing in the snow is fun,
3,Facebook (pick up the baby).mp3,44100,1,2.51,CASES,1,pick up the baby,
4,"ACPA oh let me see, what can I tell you about ...",44100,1,4.13,CASES,1,"oh let me see, what can I tell you about them now",
...,...,...,...,...,...,...,...,...
179,Video 1_6 (good).mp3,44100,2,0.76,CONTROLS,0,good,
180,Video 1_4 (and can I have some more material).mp3,44100,2,3.60,CONTROLS,0,and can I have some more material,
181,Video 1_5 (do you have a knife).mp3,44100,2,1.70,CONTROLS,0,do you have a knife,
182,Video 1_7 (here_s some pizza).mp3,44100,2,2.12,CONTROLS,0,here's some pizza,


In [ ]:
import os
from IPython.display import Audio

# Define the folder path
folder_path = '/content/drive/MyDrive/VUMC_postcleft/PUBLIC SAMPLES/CASES'

# List all files in the directory
files = os.listdir(folder_path)

# Filter out MP3 files
mp3_files = [file for file in files if file.endswith('.mp3')]

# Define a function to play audio with descriptions
def play_audio_with_description(file_name):
    file_path = os.path.join(folder_path, file_name)
    print(f"Playing file: {file_name}\n\n")  # Display the file name or any other description
    display(Audio(file_path))

# Example: Play a specific file with its description
audiofile1 = play_audio_with_description(mp3_files[70])

Playing file: Video 6_6 (bobby 2).mp3




In [ ]:
import os
from IPython.display import Audio

# Define the folder path
folder_path = '/content/drive/MyDrive/VUMC_postcleft/PUBLIC SAMPLES/CONTROLS'

# List all files in the directory
files = os.listdir(folder_path)

# Filter out MP3 files
mp3_files = [file for file in files if file.endswith('.mp3')]

# Define a function to play audio with descriptions
def play_audio_with_description(file_name):
    file_path = os.path.join(folder_path, file_name)
    print(f"Playing file: {file_name}\n\n")  # Display the file name or any other description
    display(Audio(file_path))

# Example: Play a specific file with its description
audiofile1 = play_audio_with_description(mp3_files[70])

# Final database

In [ ]:
# Define the file path
file_path = '/content/drive/MyDrive/VUMC_postcleft/output_filename.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)
df

,File_Name,Sampling_Rate_(Hz),Channels,Duration_(seconds),folder,hypernasality,original_text,OPENAI_Whisper_text
0,"ACPA 2 more in grammar school, and out little ...",44100,1,3.27,CASES,1,"2 more in grammar school, and out little 3 yea...",We're in grammar school and I'm a three-year-o...
1,ACPA and I started out on the switch board and...,44100,1,3.89,CASES,1,and I started out on the switch board and two ...,And I started out on a switchboard answering p...
2,ACPA playing in the snow is fun-3.mp3,48000,1,2.18,CASES,1,playing in the snow is fun,Playing in the snow is fun.
3,Facebook (pick up the baby).mp3,44100,1,2.51,CASES,1,pick up the baby,Set up the maze!
4,"ACPA oh let me see, what can I tell you about ...",44100,1,4.13,CASES,1,"oh let me see, what can I tell you about them now","Oh, well, let's see, what can I tell you about..."
...,...,...,...,...,...,...,...,...
179,Video 1_6 (good).mp3,44100,2,0.76,CONTROLS,0,good,Lunch
180,Video 1_4 (and can I have some more material).mp3,44100,2,3.60,CONTROLS,0,and can I have some more material,And can I have some more material?
181,Video 1_5 (do you have a knife).mp3,44100,2,1.70,CONTROLS,0,do you have a knife,Can you hand me your knife?
182,Video 1_7 (here_s some pizza).mp3,44100,2,2.12,CONTROLS,0,here's some pizza,Here's your pizza.


In [ ]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into train and test sets: stratified
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['hypernasality'], random_state=42)
print(f"Train DataFrame shape: {train_df.shape}")
print(f"Test DataFrame shape: {test_df.shape}")

Train DataFrame shape: (147, 8)
Test DataFrame shape: (37, 8)


In [ ]:
train_df
sorted_df = train_df.sort_values(by='hypernasality')
sorted_df

,File_Name,Sampling_Rate_(Hz),Channels,Duration_(seconds),folder,hypernasality,original_text,OPENAI_Whisper_text
182,Video 1_7 (here_s some pizza).mp3,44100,2,2.12,CONTROLS,0,here's some pizza,Here's your pizza.
150,video 1 (fire).mp3,44100,2,0.84,CONTROLS,0,fire,Sorry
153,NEW - video 2 (they don_t bite me either).mp3,44100,2,1.78,CONTROLS,0,they don't bite me either,They don't bite me either
179,Video 1_6 (good).mp3,44100,2,0.76,CONTROLS,0,good,Lunch
140,video 1 (four).mp3,44100,2,1.04,CONTROLS,0,four,Voila!
...,...,...,...,...,...,...,...,...
83,Video 6_3 (puppy will pull a rope).mp3,44100,2,4.55,CASES,1,puppy will pull a rope,Make me wear more wool.
90,Video 3_7 (tie and hat to eat dinner).mp3,44100,2,2.95,CASES,1,tie and hat to eat dinner,Tie and hat to eat dinner.
22,ACPA ted had a dog with white feet-4.mp3,44100,1,3.29,CASES,1,ted had a dog with white feet,That had a nose with white...
67,Video 2_10 (I ride a choo-choo train).mp3,44100,2,2.04,CASES,1,I ride a choo-choo train,I'm learning Nyeh Nyeh Nyeh.


In [ ]:
# Filter rows where 'hypernasality' is 0
zero_hypernasality_rows = sorted_df[sorted_df['hypernasality'] == 0]

# Print out 'hypernasality: 0', 'original_text', and 'OPENAI_Whisper_text' for these rows
for _, row in zero_hypernasality_rows.iterrows():
    print(f"{{hypernasality: 0, original_text: {row['original_text']}, OPENAI_Whisper_text: {row['OPENAI_Whisper_text']}}}")

In [ ]:
# Filter rows where 'hypernasality' is 0
one_hypernasality_rows = sorted_df[sorted_df['hypernasality'] == 1]

# Print out 'hypernasality: 1', 'original_text', and 'OPENAI_Whisper_text' for these rows
for _, row in one_hypernasality_rows.iterrows():
    print(f"{{hypernasality: 1, original_text: {row['original_text']}, OPENAI_Whisper_text: {row['OPENAI_Whisper_text']}}}")

In [ ]:
test_df

In [ ]:
# Filter rows where 'hypernasality' is 0
one_hypernasality_rows = test_df[test_df['hypernasality'] == 1]

# Print out 'hypernasality: 1', 'original_text', and 'OPENAI_Whisper_text' for these rows
for _, row in one_hypernasality_rows.iterrows():
    print(f"{{hypernasality: 1, original_text: {row['original_text']}, OPENAI_Whisper_text: {row['OPENAI_Whisper_text']}}}")

{hypernasality: 1, original_text: shoe, OPENAI_Whisper_text: Ew. }
{hypernasality: 1, original_text: sixty, sixty, sixty, OPENAI_Whisper_text: Hit me, hit me, hit me. }
{hypernasality: 1, original_text: we shouldn't play in the street, OPENAI_Whisper_text: Vesnyord on 8. }
{hypernasality: 1, original_text: I want a picture of giant man, OPENAI_Whisper_text: I want a picture of Dino Man. }
{hypernasality: 1, original_text: Jack likes cheese sandwiches for lunch, OPENAI_Whisper_text: I just like cheese sandwiches so much. }
{hypernasality: 1, original_text: go get the wagon, OPENAI_Whisper_text: Onion, no onion. }
{hypernasality: 1, original_text: we shouldn't play in the street, OPENAI_Whisper_text: We shouldn't play in the street. }
{hypernasality: 1, original_text: ma, ma, ma, OPENAI_Whisper_text: Maw, maw, maw. }
{hypernasality: 1, original_text: shiny shoes, OPENAI_Whisper_text: I miss you. }
{hypernasality: 1, original_text: I see the sun in the sky, OPENAI_Whisper_text: I'm gonna 

In [ ]:
# Filter rows where 'hypernasality' is 0
zero_hypernasality_rows = test_df[test_df['hypernasality'] == 0]

# Print out 'hypernasality: 0', 'original_text', and 'OPENAI_Whisper_text' for these rows
for _, row in zero_hypernasality_rows.iterrows():
    print(f"{{hypernasality: 0, original_text: {row['original_text']}, OPENAI_Whisper_text: {row['OPENAI_Whisper_text']}}}")

{hypernasality: 0, original_text: door and four, OPENAI_Whisper_text: door and... four. }
{hypernasality: 0, original_text: tires, OPENAI_Whisper_text: Tired }
{hypernasality: 0, original_text: bye-bye, OPENAI_Whisper_text: Buh-bye! }
{hypernasality: 0, original_text: where'd dad go, OPENAI_Whisper_text: Potato! }
{hypernasality: 0, original_text: do you like milk in your tea', OPENAI_Whisper_text: Do you like American yeti? }
{hypernasality: 0, original_text: don't open it, its locked, OPENAI_Whisper_text: Don't, don't open it, it's locked. }
{hypernasality: 0, original_text: And I'll put some, OPENAI_Whisper_text: Yeah, I think so. }
{hypernasality: 0, original_text: the pizza, OPENAI_Whisper_text: The pizza }
{hypernasality: 0, original_text: I say hello to spiders because they're nice to me, OPENAI_Whisper_text: I say hello to spiders because they're nice to me. }
{hypernasality: 0, original_text: papi are you ready, OPENAI_Whisper_text: Puppy, are you ready? }
{hypernasality: 0, o

# GPT4 turbo

## Failed version with gpt4 incontext

In [ ]:
import openai

# Set your API key here
openai.api_key = 'apikey'

def incontext_slp():
    # Get the definition for the specified question type
    client = openai.OpenAI(api_key='apikey')


    # Call the OpenAI API using the updated method
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        max_tokens= 4000,
        messages=[
            {"role": "assistant", "content": f"You are a professional speech pathologist trying to identify hypernasality. Here a two examples of patient recordings with hypernasality: {audiofile1}. "},
            {"role": "user", "content": f"Please tell me if this has hypernasality: {audiofile1}"}
        ]
    )

    # Extract and return the response
    #output = response['choices'][0]['message']['content']
    output = response.choices[0].message.content
    return output

output = incontext_slp()
output

"As an AI developed by OpenAI, I'm currently unable to listen to or analyze audio recordings directly. However, I can certainly guide you on how to identify hypernasality in speech.\n\nHypernasality occurs when there is an abnormal amount of nasal resonance during speech usually due to velopharyngeal dysfunction. This can happen if there is an issue with the closure of the velopharyngeal valve (the soft palate and the back of the throat) during speech, allowing air to escape through the nose excessively.\n\nHere are a few indicators that can suggest hypernasality:\n\n1. **Nasal Emissions**: Sounds of air escaping through the nose during speech, especially on high-pressure consonants like “p”, “b”, “t”, “d”, “k”, and “g”.\n\n2. **Nasalized Vowels**: Vowels sound as though they are being pronounced through the nose.\n\n3. **Reduced Oral Pressure**: Because air escapes through the nose, sounds requiring high oral pressure may sound weak or be difficult for the speaker to produce.\n\n4. **

# Whisper + GPT4 turbo incontext strategy

In [ ]:
train_prompt = """
Here are the pairs of examples that do not have hypernasality:
{hypernasality: 0, original_text: here's some pizza, OPENAI_Whisper_text: Here's your pizza. }
{hypernasality: 0, original_text: fire, OPENAI_Whisper_text: Sorry }
{hypernasality: 0, original_text: they don't bite me either, OPENAI_Whisper_text: They don't bite me either }
{hypernasality: 0, original_text: good, OPENAI_Whisper_text: Lunch }
{hypernasality: 0, original_text: four, OPENAI_Whisper_text: Voila! }
{hypernasality: 0, original_text: red, OPENAI_Whisper_text: Bread? }
{hypernasality: 0, original_text: where is it, OPENAI_Whisper_text: Where is it? }
{hypernasality: 0, original_text: he won't fly away, OPENAI_Whisper_text: He won't fly away! }
{hypernasality: 0, original_text: Ted had a dog with white feet, OPENAI_Whisper_text: Ted had a dog with white feet. }
{hypernasality: 0, original_text: can I see', OPENAI_Whisper_text: 可惜 }
{hypernasality: 0, original_text: they feed seeds to birds, OPENAI_Whisper_text: They feed seeds to birds. }
{hypernasality: 0, original_text: I played with my toys, OPENAI_Whisper_text: I played with my toys. }
{hypernasality: 0, original_text: puppies, OPENAI_Whisper_text: Puppies! }
{hypernasality: 0, original_text: yes I'm going to give him some coffee for me and him, OPENAI_Whisper_text: Yes, I'm going to make him some coffee for me and him. }
{hypernasality: 0, original_text: I like school, OPENAI_Whisper_text: I like school. }
{hypernasality: 0, original_text: and then he ate breakfast, OPENAI_Whisper_text: And then he, he ate breakfast. }
{hypernasality: 0, original_text: cookies are good to eat, OPENAI_Whisper_text: Cookies are good to eat. }
{hypernasality: 0, original_text: and where's the corn sauce, OPENAI_Whisper_text: And where's the corn sauce? }
{hypernasality: 0, original_text:  I swung on the swings and did other stuff, OPENAI_Whisper_text: I swung on the swings and did other stuff on the park. }
{hypernasality: 0, original_text: its broke, I broke it, OPENAI_Whisper_text: It broke. I broke it. }
{hypernasality: 0, original_text: where's the burner, OPENAI_Whisper_text: Foarte bună! }
{hypernasality: 0, original_text: Santa came home since the snow fell, OPENAI_Whisper_text: Santa came home since the snow fell. }
{hypernasality: 0, original_text: that's a ladybug house, right', OPENAI_Whisper_text: That's the neighbor's house, right? }
{hypernasality: 0, original_text: they just sit on the table and I say hi to them, OPENAI_Whisper_text: They just sit around the table and I say hi to them. }
{hypernasality: 0, original_text: Do you have a brother or a sister, OPENAI_Whisper_text: Do you have a brother or a sister? }
{hypernasality: 0, original_text: went to the park, OPENAI_Whisper_text: Went to the park }
{hypernasality: 0, original_text: do you have a knife, OPENAI_Whisper_text: Can you hand me your knife? }
{hypernasality: 0, original_text: we got him, OPENAI_Whisper_text: We got him! }
{hypernasality: 0, original_text: Santa came home since the snow fell, OPENAI_Whisper_text: Santa came home since the snow fell. }
{hypernasality: 0, original_text: say just hello for pretend, OPENAI_Whisper_text: Say just hello for pretend. }
{hypernasality: 0, original_text: but ants do that sometimes, OPENAI_Whisper_text: But ants do that sometimes. }
{hypernasality: 0, original_text: but, OPENAI_Whisper_text: But... }
{hypernasality: 0, original_text: Playing in the snow is fun, OPENAI_Whisper_text: Playing in the snow is fun! }
{hypernasality: 0, original_text: the polar bears are dancing, OPENAI_Whisper_text: Um, the polar bears are dancing. }
{hypernasality: 0, original_text: thank you ray, OPENAI_Whisper_text: Think away }
{hypernasality: 0, original_text: stop, OPENAI_Whisper_text: Stop! }
{hypernasality: 0, original_text: fill up the gas, OPENAI_Whisper_text: I love to guess. }
{hypernasality: 0, original_text: sue roasted a duck for supper, OPENAI_Whisper_text: Sue roasted a duck for supper. }
{hypernasality: 0, original_text: can I have your phone, OPENAI_Whisper_text: Can I have the phone? }
{hypernasality: 0, original_text: its all of our birthdays, OPENAI_Whisper_text: It's Alma's birthday. }
{hypernasality: 0, original_text: green, OPENAI_Whisper_text: Green. }
{hypernasality: 0, original_text: shh, OPENAI_Whisper_text: Shh }
{hypernasality: 0, original_text: like drums, OPENAI_Whisper_text: They drums. }
{hypernasality: 0, original_text: can I play with Jack, OPENAI_Whisper_text: Can I play with Jack? }
{hypernasality: 0, original_text: yes I am going to give him coffee for me and him, OPENAI_Whisper_text: Yes, I'm going to make him some coffee for me and him. }
{hypernasality: 0, original_text: bugs and spiders, I protect, OPENAI_Whisper_text: Bugs and spiders, I protect. }
{hypernasality: 0, original_text: pretend it stops running when the car is going, OPENAI_Whisper_text: When it stops running, when the car is going. }
{hypernasality: 0, original_text: puppy are you ready', OPENAI_Whisper_text: Puppy, are you ready? }
{hypernasality: 0, original_text: it stops going, OPENAI_Whisper_text: It stops going. }
{hypernasality: 0, original_text: shh, OPENAI_Whisper_text: Shhh! }
{hypernasality: 0, original_text: pizza bundt, OPENAI_Whisper_text: Pizza Funt! }
{hypernasality: 0, original_text: We shouldn't play in the street, OPENAI_Whisper_text: We shouldn't play in the street. }
{hypernasality: 0, original_text: I said thank you ray, OPENAI_Whisper_text: I say game great! }
{hypernasality: 0, original_text: no they dont talk, OPENAI_Whisper_text: No, they don't talk. }
{hypernasality: 0, original_text: and then he brushed his teeth, OPENAI_Whisper_text: And then he brushed his teeth. }
{hypernasality: 0, original_text: three times, OPENAI_Whisper_text: Three times. }
{hypernasality: 0, original_text: and can I have some more material, OPENAI_Whisper_text: And can I have some more material? }
{hypernasality: 0, original_text: and then go to school, OPENAI_Whisper_text: and then go to school. }
{hypernasality: 0, original_text: look at this book with us, OPENAI_Whisper_text: Look at this book with us. }
{hypernasality: 0, original_text: she wore blue shoes, OPENAI_Whisper_text: She wore blue shoes. }
{hypernasality: 0, original_text: and then he was a boy, OPENAI_Whisper_text: And then he was a boy. }
{hypernasality: 0, original_text: where's my bowl, OPENAI_Whisper_text: Where's my ball? }
{hypernasality: 0, original_text: I like school, OPENAI_Whisper_text: I like school. }
{hypernasality: 0, original_text: blue, OPENAI_Whisper_text: Blue. }
{hypernasality: 0, original_text: cookies are good to eat, OPENAI_Whisper_text: Cookies are good to eat. }
{hypernasality: 0, original_text: play some games, OPENAI_Whisper_text: Place your hands in the, }
{hypernasality: 0, original_text: yellow, OPENAI_Whisper_text: Yellow }
{hypernasality: 0, original_text: who, who, OPENAI_Whisper_text: Who? Who? }
{hypernasality: 0, original_text: but if its not, then we would stop running, OPENAI_Whisper_text: What if it's not? Then we would stop running. }
{hypernasality: 0, original_text: Most Boys Like to Play Football, OPENAI_Whisper_text: Most boys like to play football. }

Here are the pairs of examples that have hypernasality
{hypernasality: 1, original_text: do you have a brother or a sister, OPENAI_Whisper_text: Can you have a brother or it hurt? }
{hypernasality: 1, original_text: we shouldn't play in the street, OPENAI_Whisper_text: I shouldn't play with food. }
{hypernasality: 1, original_text: buy baby a bib, OPENAI_Whisper_text: Hi, I'm Hayley Mim. }
{hypernasality: 1, original_text: do you have a brother or a sister, OPENAI_Whisper_text: Do you have a brother or sister? }
{hypernasality: 1, original_text: go get the cake, OPENAI_Whisper_text: No, it's not safe. }
{hypernasality: 1, original_text: buy bobby a puppy, OPENAI_Whisper_text: Bye mommy and bubby! }
{hypernasality: 1, original_text: puppy, OPENAI_Whisper_text: Fuck me! }
{hypernasality: 1, original_text: susie sees the scissors, OPENAI_Whisper_text: blue, black, white, and blue }
{hypernasality: 1, original_text: bobby, OPENAI_Whisper_text: Thank you }
{hypernasality: 1, original_text: put the baby in the buggy, OPENAI_Whisper_text: With the Mimi and the monkey. }
{hypernasality: 1, original_text: don't wear your dirty, OPENAI_Whisper_text: Don't wear your jersey. }
{hypernasality: 1, original_text: take teddy to town, OPENAI_Whisper_text: Ning ning nang nang nang. }
{hypernasality: 1, original_text: we go swimming on a very hot day, OPENAI_Whisper_text: We go swimming on a very hot day. }
{hypernasality: 1, original_text: most boys like to play football, OPENAI_Whisper_text: Most boys like to play football. }
{hypernasality: 1, original_text: and I started out on the switch board and two phones, OPENAI_Whisper_text: And I started out on a switchboard answering phones. }
{hypernasality: 1, original_text: pamper, OPENAI_Whisper_text: Help me }
{hypernasality: 1, original_text: take a teddybear, OPENAI_Whisper_text: Hey, I'm Danny Mayo. }
{hypernasality: 1, original_text: playing in the snow is fun, OPENAI_Whisper_text: Playing a song is fun. }
{hypernasality: 1, original_text: most boys like to play football, OPENAI_Whisper_text: Vote for his right to play football. }
{hypernasality: 1, original_text: I struggle with my 's', OPENAI_Whisper_text: I struggle with my XTs. }
{hypernasality: 1, original_text: sissy sees the sun, OPENAI_Whisper_text: Я тебе не верю, Альф! }
{hypernasality: 1, original_text: ca, ca, ca, OPENAI_Whisper_text: pa-pa-pa-pa-pa }
{hypernasality: 1, original_text: take a turtle, OPENAI_Whisper_text: T-R-E-A-T-O-N-A }
{hypernasality: 1, original_text: told, OPENAI_Whisper_text: 사과하냐? }
{hypernasality: 1, original_text: but it will help me with the 's', OPENAI_Whisper_text: But it will help me with the essays. }
{hypernasality: 1, original_text: take a tire, OPENAI_Whisper_text: See you next time! }
{hypernasality: 1, original_text: go get the car, OPENAI_Whisper_text: 何だった? }
{hypernasality: 1, original_text: ted had a dog with white feet, OPENAI_Whisper_text: Ted and a dog with white feet. }
{hypernasality: 1, original_text: daddy, OPENAI_Whisper_text: Fanny }
{hypernasality: 1, original_text: Nick's grandmother lives in the city, OPENAI_Whisper_text: This is Grandmother. Have a nice day. }
{hypernasality: 1, original_text: we went to town yesterday, OPENAI_Whisper_text: We went to town yesterday }
{hypernasality: 1, original_text: Nick's grandmother lives in the city, OPENAI_Whisper_text: Next time I will go to Simi City. }
{hypernasality: 1, original_text: she went shopping, OPENAI_Whisper_text: You won't get me. }
{hypernasality: 1, original_text: in the safe deposit department, OPENAI_Whisper_text: and the safe deposit department. }
{hypernasality: 1, original_text: I like ice cream, OPENAI_Whisper_text: I like ice cream. }
{hypernasality: 1, original_text: transferring, getting information to customers, OPENAI_Whisper_text: Transferring, giving information to customers. }
{hypernasality: 1, original_text: we shouldn't play in the street, OPENAI_Whisper_text: Because I couldn't find the street. }
{hypernasality: 1, original_text: whatever they need, OPENAI_Whisper_text: Whatever they need. }
{hypernasality: 1, original_text: puffy, OPENAI_Whisper_text: funny }
{hypernasality: 1, original_text: playing in the snow is fun, OPENAI_Whisper_text: Playing in the snow is fun. }
{hypernasality: 1, original_text: see the seesaw, OPENAI_Whisper_text: See you next week, y'all! }
{hypernasality: 1, original_text: opening and closing boxes, OPENAI_Whisper_text: Opening and closing boxes. }
{hypernasality: 1, original_text: Tom had ham and eggs for breakfast, OPENAI_Whisper_text: Tom has ham and eggs for breakfast. }
{hypernasality: 1, original_text: muddy, OPENAI_Whisper_text: money }
{hypernasality: 1, original_text: pick up the baby, OPENAI_Whisper_text: Set up the maze! }
{hypernasality: 1, original_text: muddy, OPENAI_Whisper_text: money }
{hypernasality: 1, original_text: buy baby a bib, OPENAI_Whisper_text: Mad... Men... Don't...  }
{hypernasality: 1, original_text: Tom had ham and eggs for breakfast, OPENAI_Whisper_text: Tom had ham and eggs for breakfast. }
{hypernasality: 1, original_text: pick up the pie, OPENAI_Whisper_text: Sit up real high. }
{hypernasality: 1, original_text: pa, pa, pa, OPENAI_Whisper_text: Ha, ha, ha. }
{hypernasality: 1, original_text: sha, sha, sha, OPENAI_Whisper_text: Sa-sa-sa-sa }
{hypernasality: 1, original_text: ted and todd, OPENAI_Whisper_text: Ted and Todd. }
{hypernasality: 1, original_text: bah, OPENAI_Whisper_text: 来了 }
{hypernasality: 1, original_text: susie sees the dress, OPENAI_Whisper_text: I want to eat no more. }
{hypernasality: 1, original_text: One, Two, Three, Four, Five, Six, Seven, Eight, Nine, OPENAI_Whisper_text: 1, 2, 3, 4, 5, 6, 7, 8, 9 }
{hypernasality: 1, original_text: bah, bah, bah, OPENAI_Whisper_text: MWAH! MWAH! MWAH! MWAH! }
{hypernasality: 1, original_text: most boys like to play football, OPENAI_Whisper_text: That's why I like to play football. }
{hypernasality: 1, original_text: playing in the snow is fun, OPENAI_Whisper_text: Why isn't it noted? Why? }
{hypernasality: 1, original_text: my other son is a sophomore in high school, OPENAI_Whisper_text: And our next one is a sophomore in high school. }
{hypernasality: 1, original_text: do it for daddy, OPENAI_Whisper_text: Not my nanny. }
{hypernasality: 1, original_text: smashing, OPENAI_Whisper_text: Spicy! }
{hypernasality: 1, original_text: playing in the snow is fun, OPENAI_Whisper_text: Playing in the snow is fun. }
{hypernasality: 1, original_text:  Three, Four, Five, Six, Seven, Eight, Nine, OPENAI_Whisper_text: 3 4 5 6 7 8 9 }
{hypernasality: 1, original_text: feet, OPENAI_Whisper_text: Peace. }
{hypernasality: 1, original_text: well it will help me, OPENAI_Whisper_text: Wow, em vừa học đĩa }
{hypernasality: 1, original_text: pick up the books, OPENAI_Whisper_text: Hej då, mamma! }
{hypernasality: 1, original_text: go get the cookie, OPENAI_Whisper_text: Don't nick that puppy! }
{hypernasality: 1, original_text: mom, OPENAI_Whisper_text: MMMMMMMMMMMMMMMMMMMMMMMM }
{hypernasality: 1, original_text: Popeye, OPENAI_Whisper_text: Buh-bye! }
{hypernasality: 1, original_text: susie sees the horse, OPENAI_Whisper_text: Don't need to eat no more. }
{hypernasality: 1, original_text: baby, OPENAI_Whisper_text: Be me }
{hypernasality: 1, original_text: one is in school in Arizona, his father is going out there this weekend to check on him, OPENAI_Whisper_text: One is in school in Arizona, and his father is going out there this weekend to check on him. }
{hypernasality: 1, original_text: puppy will pull a rope, OPENAI_Whisper_text: Make me wear more wool. }
{hypernasality: 1, original_text: tie and hat to eat dinner, OPENAI_Whisper_text: Tie and hat to eat dinner. }
{hypernasality: 1, original_text: ted had a dog with white feet, OPENAI_Whisper_text: That had a nose with white... }
{hypernasality: 1, original_text: I ride a choo-choo train, OPENAI_Whisper_text: I'm learning Nyeh Nyeh Nyeh. }
{hypernasality: 1, original_text: seesaw, OPENAI_Whisper_text: P.S.A. }

"""

In [ ]:
# First 5 are 1, the last 5 are 0
test_prompt = """
{hypernasality: , original_text: shoe, OPENAI_Whisper_text: Ew. }
{hypernasality: , original_text: sixty, sixty, sixty, OPENAI_Whisper_text: Hit me, hit me, hit me. }
{hypernasality: , original_text: we shouldn't play in the street, OPENAI_Whisper_text: Vesnyord on 8. }
{hypernasality: , original_text: I want a picture of giant man, OPENAI_Whisper_text: I want a picture of Dino Man. }
{hypernasality: , original_text: Jack likes cheese sandwiches for lunch, OPENAI_Whisper_text: I just like cheese sandwiches so much. }
{hypernasality: , original_text: door and four, OPENAI_Whisper_text: door and... four. }
{hypernasality: , original_text: tires, OPENAI_Whisper_text: Tired }
{hypernasality: , original_text: bye-bye, OPENAI_Whisper_text: Buh-bye! }
{hypernasality: , original_text: where'd dad go, OPENAI_Whisper_text: Potato! }
{hypernasality: , original_text: do you like milk in your tea', OPENAI_Whisper_text: Do you like American yeti? }
"""

In [ ]:
import openai

# Set your API key here
openai.api_key = 'apikey'

def incontext_slp():
    # Get the definition for the specified question type
    client = openai.OpenAI(api_key='apikey')


    # Call the OpenAI API using the updated method
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        max_tokens= 4000,
        messages=[
            {"role": "assistant", "content": f"You are a professional speech pathologist trying to identify hypernasality. Here is the data: {train_prompt}"},
            {"role": "user", "content": f"Please tell me if these examples have hypernasality by filling in 0 for no hypernasality and 1 for hypernasality: \n{test_prompt}"}
        ]
    )

    # Extract and return the response
    #output = response['choices'][0]['message']['content']
    output = response.choices[0].message.content
    return output

output = incontext_slp()
output

"{hypernasality: 1, original_text: shoe, OPENAI_Whisper_text: Ew. }\n{hypernasality: 1, original_text: sixty, sixty, sixty, OPENAI_Whisper_text: Hit me, hit me, hit me. }\n{hypernasality: 1, original_text: we shouldn't play in the street, OPENAI_Whisper_text: Vesnyord on 8. }\n{hypernasality: 0, original_text: I want a picture of giant man, OPENAI_Whisper_text: I want a picture of Dino Man. }\n{hypernasality: 0, original_text: Jack likes cheese sandwiches for lunch, OPENAI_Whisper_text: I just like cheese sandwiches so much. }\n{hypernasality: 0, original_text: door and four, OPENAI_Whisper_text: door and... four. }\n{hypernasality: 1, original_text: tires, OPENAI_Whisper_text: Tired }\n{hypernasality: 0, original_text: bye-bye, OPENAI_Whisper_text: Buh-bye! }\n{hypernasality: 1, original_text: where'd dad go, OPENAI_Whisper_text: Potato! }\n{hypernasality: 1, original_text: do you like milk in your tea', OPENAI_Whisper_text: Do you like American yeti? }"

In [ ]:
print(output)

{hypernasality: 1, original_text: shoe, OPENAI_Whisper_text: Ew. }
{hypernasality: 1, original_text: sixty, sixty, sixty, OPENAI_Whisper_text: Hit me, hit me, hit me. }
{hypernasality: 1, original_text: we shouldn't play in the street, OPENAI_Whisper_text: Vesnyord on 8. }
{hypernasality: 0, original_text: I want a picture of giant man, OPENAI_Whisper_text: I want a picture of Dino Man. }
{hypernasality: 0, original_text: Jack likes cheese sandwiches for lunch, OPENAI_Whisper_text: I just like cheese sandwiches so much. }
{hypernasality: 0, original_text: door and four, OPENAI_Whisper_text: door and... four. }
{hypernasality: 1, original_text: tires, OPENAI_Whisper_text: Tired }
{hypernasality: 0, original_text: bye-bye, OPENAI_Whisper_text: Buh-bye! }
{hypernasality: 1, original_text: where'd dad go, OPENAI_Whisper_text: Potato! }
{hypernasality: 1, original_text: do you like milk in your tea', OPENAI_Whisper_text: Do you like American yeti? }


# First accuracy: 5/10 -> 50%